# Starting Lineup Converter

In [2]:
from basketball_reference_web_scraper import client
import pandas as pd
import numpy as np

## Make **'PLAYER_DICT'**

In [115]:
def get_ids(year):
    df = pd.DataFrame(client.players_season_totals(year))
    new_df = df[['name','slug']].copy()
    return new_df

def get_all_ids(year_start, year_end):
    df = pd.concat([get_ids(year) for year in range(year_start, year_end+1)])
    df.drop_duplicates(inplace=True)
    return df

In [116]:
all_players = get_all_ids(2012,2020)

In [118]:
all_players.describe()

,name,slug
count,1237,1237
unique,1233,1237
top,Tony Mitchell,maxieja01
freq,2,1


In [132]:
# name_index = all_players
name = all_players["name"].str.split(" ", n = -1, expand = True)
name.columns = ['first_name','last_name', 'other_1','other_2']
name.drop(columns=['other_1','other_2'], inplace=True)
name['Lineup_name'] = name['first_name'].str[0]+'. '+ name['last_name'].str[:]

lineup_df = pd.concat([all_players, name], axis='columns')
lineup_df.drop(columns=['first_name','last_name','slug'], inplace=True)
lineup_df.head()

,name,Lineup_name
0,Jeff Adrien,J. Adrien
1,Arron Afflalo,A. Afflalo
2,Blake Ahearn,B. Ahearn
3,Solomon Alabi,S. Alabi
4,Cole Aldrich,C. Aldrich


In [133]:
lineup_df.index = lineup_df['Lineup_name']
lineup_df.drop(columns=['Lineup_name'],inplace=True)
PLAYER_DICT = lineup_df.T.to_dict('list')

C:\Users\ekuo651\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until


In [185]:
len(PLAYER_DICT)

1149

In [137]:
def lineup_name_converter(string):
    name = PLAYER_DICT[string]
    return name

In [136]:
name_converter('L. James')

['LeBron James']

## Make **'starter_fetcher'**

In [83]:
starting_lineup = pd.read_csv('../Resources/sample_lineup.csv')
starting_lineup.head()

,Unnamed: 0,TEAM,PG,SG,SF,PF,C
0,0,Atlanta,T. Young,K. Huerter,D. Hunter,J. Collins,C. Capela
1,1,Boston,K. Walker,J. Brown,J. Tatum,G. Hayward,D. Theis
2,2,Brooklyn,K. Irving,J. Harris,C. LeVert,T. Prince,J. Allen
3,3,Charlotte,D. Graham,T. Rozier,M. Bridges,P. Washington,C. Zeller
4,4,Chicago,T. Satoransky,Z. LaVine,K. Dunn,L. Markkanen,L. Kornet


In [84]:
starting_lineup.TEAM = starting_lineup.TEAM.str.upper()

In [85]:
starting_lineup.index = starting_lineup.TEAM
starting_lineup = starting_lineup[['PG','SG', 'SF','PF','C']]

In [99]:
starting_lineup.head()

,PG,SG,SF,PF,C
TEAM,,,,,
ATLANTA,T. Young,K. Huerter,D. Hunter,J. Collins,C. Capela
BOSTON,K. Walker,J. Brown,J. Tatum,G. Hayward,D. Theis
BROOKLYN,K. Irving,J. Harris,C. LeVert,T. Prince,J. Allen
CHARLOTTE,D. Graham,T. Rozier,M. Bridges,P. Washington,C. Zeller
CHICAGO,T. Satoransky,Z. LaVine,K. Dunn,L. Markkanen,L. Kornet


In [106]:
starting_lineup = starting_lineup.T

In [184]:
starting_lineup.columns

Index(['ATLANTA', 'BOSTON', 'BROOKLYN', 'CHARLOTTE', 'CHICAGO', 'CLEVELAND',
       'DALLAS', 'DENVER', 'DETROIT', 'GOLDEN STATE', 'HOUSTON', 'INDIANA',
       'LA CLIPPERS', 'LA LAKERS', 'MEMPHIS', 'MIAMI', 'MILWAUKEE',
       'MINNESOTA', 'NEW ORLEANS', 'NEW YORK', 'OKLAHOMA CITY', 'ORLANDO',
       'PHILADELPHIA', 'PHOENIX', 'PORTLAND', 'SACRAMENTO', 'SAN ANTONIO',
       'TORONTO', 'UTAH', 'WASHINGTON'],
      dtype='object', name='TEAM')

dict_values

## Make **'CURR_DICT'**

In [138]:
import sys
sys.path.append('../Scripts')

import scheduling_functions as sfx

In [139]:
schedule = pd.read_csv('../Resources/game_schedule_13-current.csv')
clean_schedule = sfx.get_encoded_schedule(schedule)
clean_schedule.head()

,date1,date2,away,home,away_team_score,home_team_score,game_number,game_id
0,2012-10-30,2012-10-29,WASHINGTON_WIZARDS,CLEVELAND_CAVALIERS,84.0,94.0,0,0
1,2012-10-31,2012-10-30,BOSTON_CELTICS,MIAMI_HEAT,107.0,120.0,1,1
2,2012-10-31,2012-10-30,DALLAS_MAVERICKS,LOS_ANGELES_LAKERS,99.0,91.0,2,2
3,2012-10-31,2012-10-30,INDIANA_PACERS,TORONTO_RAPTORS,90.0,88.0,3,3
4,2012-10-31,2012-10-30,DENVER_NUGGETS,PHILADELPHIA_76ERS,75.0,84.0,4,4


In [183]:
team = clean_schedule[['away','home']].copy()
team.drop(columns=['home'],inplace=True)
team.drop_duplicates(inplace=True)
team.sort_values('away', ascending=True).reset_index()

,index,away
0,38,ATLANTA_HAWKS
1,1,BOSTON_CELTICS
2,54,BROOKLYN_NETS
3,31,CHARLOTTE_BOBCATS
4,2673,CHARLOTTE_HORNETS
5,17,CHICAGO_BULLS
6,32,CLEVELAND_CAVALIERS
7,2,DALLAS_MAVERICKS
8,4,DENVER_NUGGETS
9,22,DETROIT_PISTONS


In [148]:
all_teams = list(set(clean_schedule['away'].to_list()))

In [ ]:
team.drop(columns=['home'],inplace=True)


In [176]:
team['away'].str.strip()
team['away'] = team['away'].str.replace('_','')
team

,away
0,WASHINGTONWIZARDS
1,BOSTONCELTICS
2,DALLASMAVERICKS
3,INDIANAPACERS
4,DENVERNUGGETS
5,HOUSTONROCKETS
6,SANANTONIOSPURS
7,SACRAMENTOKINGS
9,GOLDENSTATEWARRIORS
10,MEMPHISGRIZZLIES


In [177]:
team.drop_duplicates(inplace=True)

In [178]:
team

,away
0,WASHINGTONWIZARDS
1,BOSTONCELTICS
2,DALLASMAVERICKS
3,INDIANAPACERS
4,DENVERNUGGETS
5,HOUSTONROCKETS
6,SANANTONIOSPURS
7,SACRAMENTOKINGS
9,GOLDENSTATEWARRIORS
10,MEMPHISGRIZZLIES


In [181]:
team.sort_values('away', ascending=True)

,away
38,ATLANTAHAWKS
1,BOSTONCELTICS
54,BROOKLYNNETS
31,CHARLOTTEBOBCATS
2673,CHARLOTTEHORNETS
17,CHICAGOBULLS
32,CLEVELANDCAVALIERS
2,DALLASMAVERICKS
4,DENVERNUGGETS
22,DETROITPISTONS


In [145]:
all_teams

<function list.sort(*, key=None, reverse=False)>